In [23]:
import numpy as np
import cv2
import os

In [24]:
camera_number = 1
frame_count = 0
frame_skip = 30

abs_dir_path = "/home/emanuele/ComputerVision/CV-CameraCalibration/videos/"
VIDEO_NAME = f'out{camera_number}F.mp4'
ROW_POINTS = 5
COLUMNS_POINTS = 5
# Open the video file
video_capture = cv2.VideoCapture(f'{abs_dir_path}{VIDEO_NAME}')

# Print the number of frames in the video
number_of_frame = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))
print("Number of frames in the video: ", number_of_frame)

Number of frames in the video:  2584


In [25]:
# Create a directory to save the screen captures named as the video file
output_dir = 'samples/' + VIDEO_NAME
os.makedirs(output_dir, exist_ok=True)
print("Saving frames to ", output_dir)
# Remove all files in the directory
for file in os.listdir(output_dir):
    os.remove(os.path.join(output_dir, file))



Saving frames to  samples/out1F.mp4


In [13]:


while True:
        # Read a frame from the video
        ret, img = video_capture.read()
        if not ret:
            break  # Break the loop if we've reached the end of the video

        # Skip frames based on the frame_skip value
        if frame_count % frame_skip == 0:
            # Resize the frame to the new_resolution
            real_img = img
            
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            #corners = []
            # Find the chessboard corners
            found_corners, corners = cv2.findChessboardCorners(gray, (ROW_POINTS, COLUMNS_POINTS), corners= None)
            print(found_corners)
            res_img = cv2.drawChessboardCorners(img,(ROW_POINTS,COLUMNS_POINTS),corners,found_corners)
            if found_corners:
                print(f'corners: {corners[0:3]}')
                
                cv2.namedWindow(f'frame {frame_count}', flags= cv2.WINDOW_NORMAL)
                cv2.resizeWindow(f'frame {frame_count}', 1280, 720)
                cv2.imshow(f'frame {frame_count}', res_img)
                cv2.waitKey(10000)
                cv2.destroyAllWindows()
            print(frame_count)
        
            #print("Frame ", frame_count, ":", found_corners)
            #res_img = cv2.cvtColor(res_img, cv2.COLOR_GRAY2BGR)
        frame_count += 1  
        if frame_count == 1000:
            break  

False
570
False
600


KeyboardInterrupt: 

In [32]:
video_capture = cv2.VideoCapture(f'{abs_dir_path}{VIDEO_NAME}')

FRAME_NUMBER = 200
frame_count = 0
img = 0
while frame_count != FRAME_NUMBER:
    ret, img = video_capture.read()
    frame_count += 1


cv2.namedWindow(f'{FRAME_NUMBER}', flags= cv2.WINDOW_NORMAL)
cv2.resizeWindow(f'{FRAME_NUMBER}', 1280, 720)
cv2.imshow(f'{FRAME_NUMBER}', img)
cv2.waitKey(0)

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

found_corners, corners = cv2.findChessboardCorners(gray, (ROW_POINTS, COLUMNS_POINTS), corners= None)
res_img = cv2.drawChessboardCorners(img,(ROW_POINTS,COLUMNS_POINTS),corners,found_corners)
if found_corners:
    cv2.imshow(f'{FRAME_NUMBER}', res_img)
    cv2.waitKey(10000)
cv2.destroyAllWindows()